In [1]:
%matplotlib inline
import pandas as pd
import pandasql as ps
from datetime import datetime
import seaborn
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'svg'
from pylab import rcParams
rcParams['figure.figsize'] = 8, 5

In [2]:
pd.__version__

'1.0.1'

In [3]:
project_submissions = pd.read_csv('./data/project_submissions.csv')
daily_engagements = pd.read_csv('./data/daily_engagement.csv')
enrollments = pd.read_csv('./data/enrollments.csv')

In [4]:
# pandasql code
def example1_pandasql(daily_engagements):
    simple_query = '''
        SELECT 
            acct, 
            total_minutes_visited,
            utc_date
        FROM daily_engagements 
        ORDER BY total_minutes_visited desc
        LIMIT 10
        '''
    return ps.sqldf(simple_query, locals())

example1_pandasql(daily_engagements)

,acct,total_minutes_visited,utc_date
0,317,1030.883197,2015-07-11
1,328,945.538914,2015-07-09
2,198,876.512846,2014-12-30
3,163,872.633923,2015-07-10
4,573,866.405226,2015-07-11
5,303,856.634726,2015-05-14
6,619,853.253236,2015-07-10
7,163,850.519340,2015-07-09
8,108,820.879483,2015-02-20
9,278,816.895443,2015-07-09


In [5]:
# pandas code
def example1_pandas(daily_engagements):
    return daily_engagements[['acct', 'total_minutes_visited', 'utc_date']].sort_values(by ='total_minutes_visited', ascending = False)[:10]

example1_pandas(daily_engagements)

,acct,total_minutes_visited,utc_date
54536,317,1030.883197,2015-07-11
56403,328,945.538914,2015-07-09
33728,198,876.512846,2014-12-30
27699,163,872.633923,2015-07-10
97492,573,866.405226,2015-07-11
51779,303,856.634726,2015-05-14
105968,619,853.253236,2015-07-10
27698,163,850.519340,2015-07-09
18394,108,820.879483,2015-02-20
47372,278,816.895443,2015-07-09


In [6]:
daily_engagements['weekday'] = map(lambda x: datetime.strptime(x, '%Y-%m-%d').strftime('%A'), daily_engagements.utc_date)

daily_engagements.head()

,acct,utc_date,num_courses_visited,total_minutes_visited,lessons_completed,projects_completed,weekday
0,0,2015-01-09,1.0,11.679374,0.0,0.0,<map object at 0x000002A9D92A4820>
1,0,2015-01-10,2.0,37.284887,0.0,0.0,<map object at 0x000002A9D92A4820>
2,0,2015-01-11,2.0,53.633746,0.0,0.0,<map object at 0x000002A9D92A4820>
3,0,2015-01-12,1.0,33.489270,0.0,0.0,<map object at 0x000002A9D92A4820>
4,0,2015-01-13,1.0,64.779678,0.0,0.0,<map object at 0x000002A9D92A4820>


In [9]:
# pandasql code
def example2_pandasql(daily_engagements):
    aggr_query = '''
        SELECT 
            avg(total_minutes_visited) as total_minutes_visited,
            weekday
        FROM daily_engagements 
        GROUP BY weekday
        '''
    return ps.sqldf(aggr_query, locals()).set_index('weekday')

weekday_engagement = example2_pandasql(daily_engagements)
weekday_engagement

InterfaceError: (sqlite3.InterfaceError) Error binding parameter 6 - probably unsupported type.
[SQL: INSERT INTO daily_engagements (acct, utc_date, num_courses_visited, total_minutes_visited, lessons_completed, projects_completed, weekday) VALUES (?, ?, ?, ?, ?, ?, ?)]
[parameters: ((0, '2015-01-09', 1.0, 11.6793745, 0.0, 0.0, <map object at 0x000002A9D92A4820>), (0, '2015-01-10', 2.0, 37.2848873333, 0.0, 0.0, <map object at 0x000002A9D92A4820>), (0, '2015-01-11', 2.0, 53.633746333299996, 0.0, 0.0, <map object at 0x000002A9D92A4820>), (0, '2015-01-12', 1.0, 33.489269666700004, 0.0, 0.0, <map object at 0x000002A9D92A4820>), (0, '2015-01-13', 1.0, 64.7796776667, 0.0, 0.0, <map object at 0x000002A9D92A4820>), (0, '2015-01-14', 1.0, 151.617576833, 1.0, 0.0, <map object at 0x000002A9D92A4820>), (0, '2015-01-15', 1.0, 142.395963833, 0.0, 0.0, <map object at 0x000002A9D92A4820>), (0, '2015-01-16', 0.0, 0.0, 0.0, 0.0, <map object at 0x000002A9D92A4820>)  ... displaying 10 of 136240 total boun

In [10]:
# pandas code
def example2_pandas(daily_engagements):
    return pd.DataFrame(daily_engagements.groupby('weekday').total_minutes_visited.mean())
example2_pandas(daily_engagements)

,total_minutes_visited
weekday,
<map object at 0x000002A9D92A4820>,24.55168
